In [2]:
import os
import chromadb
from openai import AzureOpenAI

chromadb_client = chromadb.CloudClient(
  api_key=os.getenv('CHROMA_API_KEY'),
  tenant=os.getenv('CHROMA_TENANT'),
  database=os.getenv('CHROMA_DATABASE')
)

azure_client = AzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT')
)

In [7]:
embedding_model ="text-embedding-3-large"

def get_embeddings(text: str):
    """Gets embeddings from Azure OpenAI"""
    response = azure_client.embeddings.create(input=text, model=embedding_model, dimensions=1024)
    embeddings = response.data[0].embedding
    return embeddings

In [ ]:
import json

SUMMARIZE_PROMPT = """A user is searching in a virtual museum. 
The user asked the following search query: {query}
Here are the results that were retrieved: {retrieval}
Generate a friendly sentence that acknowledges the query 
and reminds the user of what they might be interested in."""

DESCRIBE_AND_JUSTIFY_PROMPT = """A user is searching in a virtual museum. 
The user asked the following search query: {query}
A result was retrieved: {title}
Here is its information: {content}
Describe the result briefly and cheerfully.
Justify the result's relevance briefly."""

def describe_results(query, contents):
    """Generates a fun description of search results"""
    retrieval = '\n\n'.join(contents)
    prompt = SUMMARIZE_PROMPT.format(query=query, retrieval=retrieval)
    response = azure_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{'role': 'user', 'content': prompt}]
    )

    descriptor = response.choices[0].message.content
    return descriptor

def describe_result(query, title, content):
    """Generates a description and a justification for a result"""
    prompt = DESCRIBE_AND_JUSTIFY_PROMPT.format(query=query, title=title, content=content)
    response = azure_client.chat.completions.create(
        model='gpt-4o-mini',
        response_format={'type': 'json_object'},
        messages=[
            {'role': 'system', 'content': 'Output your JSON result as follows: {"description": your_description, "justification": your_justification}'},
            {'role': 'user', 'content': prompt}
        ]
    )

    result = json.loads(response.choices[0].message.content)
    return result['description'], result['justification']
    
# Step 1. Get the results and parse the data
query='Super super cold stuff!'
specimens = chromadb_client.get_collection('specimens')
embeddings = get_embeddings(text=query)
results = specimens.query(query_embeddings=embeddings)
contents = results['documents'][0][:5]
metadatas = results['metadatas'][0][:5]

# Step 2. Generate a descriptor of the result
general_message = describe_results(query, contents)

# Step 3. List recommendations
recommendations = []

for content, metadata in zip(contents, metadatas):
    identifier, title = metadata['specimen_name'], metadata['title']
    description, justification = describe_result(query, title, content)
    recommendations.append({
        'identifier': identifier,
        'name': title,
        'description': description,
        'justification': justification
    })

In [119]:
general_message

"It sounds like you're on the lookout for some fascinatingly cold specimens! You might find the Frozen Hawk intriguing, or perhaps the unique Baby Crocodile Mummies and the Baby Caiman will pique your interest as well—each with their own chilling story!"

In [120]:
recommendations

[{'identifier': 'RM 2019-04-07 frozen hawk',
  'name': 'Frozen hawk',
  'description': "The 'Frozen Hawk' is a fascinating specimen that perfectly embodies the theme of extreme cold! Encounter this remarkable bird preserved in ice at the Redpath Museum, a stunning testament to the beauty and resilience of nature's chillier creatures.",
  'justification': "The search query 'Super super cold stuff!' aligns well with the 'Frozen Hawk,' as it directly pertains to a unique exhibit that showcases something frozen, thus fitting the user's interest in cold-related items."},
 {'identifier': 'RM 2725-01 croc mummy',
  'name': 'Baby crocodile mummy',
  'description': "Check out this adorable Baby Crocodile Mummy! This little creature, believed to be a Crocodylus suchus, is located at the Redpath Museum. It's a fascinating glimpse into the ancient past, showcasing how incredibly preserved this 'super super cold' mummy is. Isn't it intriguing to see such unique and cool specimens?",
  'justificatio

In [ ]:

query='Super super cold stuff!'
embeddings = get_embeddings(text=query)
results = specimens.query(query_embeddings=embeddings)

documents = results['documents'][0][:5]
metadatas = results['metadatas'][0][:5]

retrieval = '\n\n'.join(documents)

In [45]:
results

{'ids': [['dcb43025-0791-43cf-ab0e-bbe15b886e2b',
   '52946054-aef2-478d-9674-df995c5a9103',
   'd0546c36-1868-4351-a4f2-a697166c7798',
   '2c54f93f-f765-43a4-90ae-064812560f17',
   '7e3d4fd0-6692-41e2-9592-316cdf013d47',
   '02ae9dfd-9871-4593-8c21-703d83275c8f',
   '3f962775-dbd2-4281-b00d-b9334b32d476',
   'cfe03260-e185-4db7-ab7f-33ddd501f38e',
   '333eebab-b2e1-4ae4-8b69-63f4c813de35',
   'a16698d6-cdba-4f6b-a6a9-9183c5f3d3b4']],
 'distances': [[1.396086,
   1.5711721,
   1.5721308,
   1.6075327,
   1.608721,
   1.6121161,
   1.6292788,
   1.6340938,
   1.639032,
   1.6553724]],
 'embeddings': None,
 'metadatas': [[{'row': 20,
    'source': '../../data/processed/vrmuseum.csv',
    'specimen_name': 'RM 2019-04-07 frozen hawk',
    'title': 'Frozen hawk'},
   {'specimen_name': 'RM 2725-01 croc mummy',
    'source': '../../data/processed/vrmuseum.csv',
    'row': 15,
    'title': 'Baby crocodile mummy'},
   {'title': 'Baby crocodile mummy',
    'row': 16,
    'source': '../../data/pr

In [49]:
SUMMARIZE_PROMPT = """A user is searching in a virtual museum. 
The user asked the following search query: {query}
Here are the results that were retrieved: {retrieval}
Generate a friendly sentence that acknowledges the query 
and reminds the user of what they might be interested in."""

In [50]:
response = azure_client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{'role': 'user', 'content': SUMMARIZE_PROMPT.format(query=query, retrieval=retrieval)}]
)

In [52]:
response.choices[0].message.content

'It sounds like you\'re curious about some really fascinating frozen specimens! You might want to explore the "Frozen Hawk" or the various mummies, including the intriguing "Falcon Mummy" and the "Baby Crocodile Mummies" we have on display. Let me know if you\'d like more information on any of these!'

In [53]:
content, title = documents[0], metadatas[0]['title']

In [ ]:
DESCRIBE_AND_JUSTIFY_PROMPT = """A user is searching in a virtual museum. 
The user asked the following search query: {query}
A result was retrieved: {title}
Here is its information: {content}
Describe the result briefly and cheerfully.
Justify the result's relevance briefly."""

In [109]:
import json

response = azure_client.chat.completions.create(
    model='gpt-4o-mini',
    response_format={'type': 'json_object'},
    messages=[
        {'role': 'system', 'content': 'Output your JSON result as follows: {"description": your_description, "justification": your_justification}'},
        {'role': 'user', 'content': DESCRIBE_AND_JUSTIFY_PROMPT.format(query=query, title=title, content=content)}
    ]
)

result = response.choices[0].message.content
generated_metadata = json.loads(result)
description, justification = generated_metadata['description'], generated_metadata['justification']

In [110]:
description


"The 'Frozen Hawk' is a captivating specimen on display at the Redpath Museum, showcasing the intriguing aspects of nature in its frigid state! It's an eye-catching example of wildlife preservation that sparks curiosity and awe. Visiting this exhibit is like stepping into a chilly world of fascinating frozen beauty!"

In [111]:
justification

"The search query 'Super super cold stuff!' aligns perfectly with the result 'Frozen hawk', as it refers to a specimen that embodies extreme cold conditions. This exhibit showcases a unique biological specimen that likely intrigues visitors interested in cold environments."